In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive

In [ ]:
cd "My Drive/Practice"

In [ ]:
#importing the modules 

import numpy as np    #numpy for mathematical operation on data
import pandas as pd    # python library to work on data
import datetime         
import os            #module of python provides function for interacting with the operating syestem.
import time             
import matplotlib.pyplot as plt   #for plotting and visual representation of data
import seaborn as sns             #visualisation library based on matplotlib
import pickle
  #plotting commands just below the code cell 
%matplotlib inline

In [5]:
#reading the The Movies Dataset (metadata for movies listed in full movie lens dataset)
md = pd.read_csv('movies_metadata.csv')
md.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
#reading  links_small  (movie id -- tmdbid)
links_small = pd.read_csv('links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
#converting tmbd id into int
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [ ]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int');
#only taking those movie which have information in links_small
smd = md[md['id'].isin(links_small)]
smd['id'] = smd['id'].astype('int');
smd.shape

In [ ]:
smd['id']

In [10]:
credits = pd.read_csv('credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [11]:
keywords = pd.read_csv('keywords.csv');
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [12]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

In [13]:
smd = smd.merge(credits, on='id')
smd = smd.merge(keywords, on='id')

In [14]:
#applying literal_eval to convert into usable python structure
from ast import literal_eval
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['genres'] = smd['genres'].apply(literal_eval);

smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [15]:
#making a list of genres from the structure 
smd['genres']=smd['genres'].fillna('[]');
smd['genres'] = smd['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [16]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [17]:
#getting director from crew list
smd['director'] = smd['crew'].apply(get_director)

In [18]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    #Return empty list in case of missing/malformed data
    return []

In [19]:
smd['cast']=smd['cast'].apply(get_list)

In [20]:
#list of keywords
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [21]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [22]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
  smd[feature] = smd[feature].apply(clean_data)

In [23]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
smd['soup'] = smd.apply(create_soup, axis=1)

In [24]:
smd = smd.reset_index()
indices = pd.Series(smd.index, index=smd['title'])

In [25]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [26]:
#saving to csv file after processing to use it in the app
smd.to_csv('smd.csv',index=False)

In [27]:
#recommend those which are movies similar to the input
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return smd['title'].iloc[movie_indices]

In [46]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim_x = cosine_similarity(count_matrix, count_matrix)

In [48]:
filename = 'cosine_similarity.sav'
#store the cosine_similarity matrix using pickle
pickle.dump(cosine_sim_x, open(filename, 'wb'))
# load the model from disk
cosine_sim_x = pickle.load(open(filename, 'rb'))

In [49]:
get_recommendations('The Dark Knight',cosine_sim_x)

8031         The Dark Knight Rises
6218                 Batman Begins
7659    Batman: Under the Red Hood
1260                Batman & Robin
1134                Batman Returns
524                         Batman
5809                   Point Blank
6645                   Harsh Times
7561                   Harry Brown
7582                      Defendor
Name: title, dtype: object

In [32]:
#Colloborative Filtering
#use SVD and surprise library to implement SVD
#install surprise library
!pip install surprise

     |████████████████████████████████| 11.8MB 3.6MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670918 sha256=4ee3ce7ed835d7127118f906767fe056531fc2d7c50e87f147b65b28b6ed1a0c
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [33]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate 
reader = Reader()

#used ratings_small from the movies data set (so that we can get users and their rating for movies)
ratings = pd.read_csv('ratings_small.csv')
print(ratings.shape)
print(ratings.head())

(100004, 4)
   userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205


In [34]:
#readerclass is used to read custom data set and parse it so that used by library function
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [35]:
algo = SVD()
# Run 5-fold cross-validation and then print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8940  0.9025  0.8890  0.9014  0.8956  0.8965  0.0050  
MAE (testset)     0.6884  0.6932  0.6864  0.6954  0.6875  0.6902  0.0035  
Fit time          4.63    4.65    4.65    4.62    4.64    4.64    0.01    
Test time         0.14    0.23    0.13    0.13    0.13    0.15    0.04    


{'fit_time': (4.6284871101379395,
  4.652105093002319,
  4.6507720947265625,
  4.619409561157227,
  4.644089698791504),
 'test_mae': array([0.68836403, 0.69320577, 0.68643073, 0.6954066 , 0.68751018]),
 'test_rmse': array([0.89403631, 0.90248696, 0.88901143, 0.90136234, 0.8955539 ]),
 'test_time': (0.13608002662658691,
  0.23235869407653809,
  0.13060426712036133,
  0.127488374710083,
  0.13070225715637207)}

In [36]:
#to fit on complete dataset
trainset = data.build_full_trainset()
algo.fit(trainset)

In [37]:
algo.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.5564554529868655, details={'was_impossible': False})

In [38]:
#content based recommender for The Movies Dataset
def convert_int(x):
  try:
    return int(x)
  except:
    return np.nan

In [39]:
#reading links_small
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]

#changed to int (tmdb Id)
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']

#merging id_map with smd on tmdbid of movie
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [40]:
#id_map with  tmdb id as index
indices_map = id_map.set_index('id')

In [43]:
import pickle
filename = 'finalized_model.sav'
#storing the SVD Model using Pickle
pickle.dump(algo, open(filename, 'wb'))
# load the model from disk
algo_new = pickle.load(open(filename, 'rb'))

In [44]:
#the hybrid recommender function which predicts the list of movies a user will like based on his/her choice
def hybrid1(userId, title):    
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim_x[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: algo_new.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [45]:
hybrid1(1,'Avatar')

,title,vote_count,vote_average,id,est
8712,Guardians of the Galaxy,10014.0,7.9,118340,3.098678
974,Aliens,3282.0,7.7,679,3.056031
1241,The Fifth Element,3962.0,7.3,18,3.030034
8626,Captain America: The Winter Soldier,5881.0,7.6,100402,2.986015
2834,Predator,2129.0,7.3,106,2.971270
7705,Alice in Wonderland,8.0,5.4,25694,2.876949
648,Independence Day,3334.0,6.7,602,2.837040
3060,Sinbad and the Eye of the Tiger,39.0,6.3,11940,2.800026
7068,Stargate: Continuum,151.0,6.8,12914,2.778476
9211,Kingsglaive: Final Fantasy XV,201.0,6.8,390734,2.776314
